
# 🏍️ MotoTrack – Colab Notebook (Simulação ponta‑a‑ponta)
**Pronto para Google Colab.** Este notebook gera telemetria simulada (IoT/Visão), aplica regras (geofence, bateria, inatividade),
visualiza no **mapa (Folium)**, e oferece um **mock SSE** para demonstrar alerta em tempo real.  
Ideal para evidências da entrega (vídeo ≤ 5 min).

> **Dica**: Não é necessário GPU. Tempo total de execução: ~1–3 min.



## 1) Preparação do Ambiente (Colab)
- Instala dependências (Folium, Pandas, etc.).
- (Opcional) Monta o Google Drive para salvar evidências no seu Drive.

> Execute esta célula primeiro.


In [ ]:

# @title Instalar dependências e (opcional) montar o Google Drive
# Se estiver no Colab, estas linhas funcionam normalmente. Em outros ambientes, ajuste conforme necessário.
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Instala pacotes (silencioso no Colab)
if IN_COLAB:
    !pip -q install pandas folium matplotlib flask
    
# Montar Drive (opcional)
if IN_COLAB:
    from google.colab import drive  # type: ignore
    try:
        drive.mount('/content/drive', force_remount=False)
        DRIVE_MOUNTED = True
    except Exception as e:
        print("Drive não montado (opcional).", e)
        DRIVE_MOUNTED = False
else:
    DRIVE_MOUNTED = False

print("IN_COLAB:", IN_COLAB, "| DRIVE_MOUNTED:", DRIVE_MOUNTED)



## 2) Configurações
Ajuste o centro e raio da geofence, quantidade de registros e aleatoriedade.


In [ ]:

# @title Parâmetros de simulação
import math, random, datetime as dt, json
from pathlib import Path

# Centro do pátio (Av. Paulista, SP por padrão)
GEOFENCE_CENTER = (-23.561684, -46.655981)  # (lat, lon)
GEOFENCE_RADIUS_M = 120.0  # raio da geofence (m)

# Telemetria
N_REGISTROS = 300            # total de amostras
P_OFF_GEOFENCE = 0.22        # % de pontos fora da geofence (gera alertas)
SEED = 42                    # reprodutibilidade
PLACAS = ["ABC1D23", "EFG4H56", "IJK7L89", "MNO0P12", "QRS3T45"]

# Saídas
OUT_DIR = Path("/content") if 'google.colab' in str(globals()) else Path("/mnt/data")
PATH_NDJSON = OUT_DIR / "telemetria.ndjson"
PATH_ALERTAS = OUT_DIR / "alertas.ndjson"
PATH_MAP_HTML = OUT_DIR / "mapa_mototrack.html"

random.seed(SEED)
print("Saídas:", PATH_NDJSON, PATH_ALERTAS, PATH_MAP_HTML, sep="\n- ")



## 3) Gerador de Telemetria
Cria pontos próximos ao pátio, com variação controlada. Exporta **NDJSON** para evidências.


In [ ]:

# @title Gerar telemetria simulada e salvar NDJSON
import pandas as pd
import math, random

def meters_offset(lat, lon, dlat_m=0, dlon_m=0):
    dlat = dlat_m / 111_320
    dlon = dlon_m / (111_320 * math.cos(math.radians(lat)))
    return lat + dlat, lon + dlon

def gen_point_near(center, max_dist_m=80):
    base_lat, base_lon = center
    dlat = random.uniform(-max_dist_m, max_dist_m)
    dlon = random.uniform(-max_dist_m, max_dist_m)
    return meters_offset(base_lat, base_lon, dlat, dlon)

def gen_telemetria(n_registros=100, p_off_geofence=0.15):
    out = []
    now = dt.datetime.utcnow()
    for i in range(n_registros):
        placa = random.choice(PLACAS)
        # 1) 78% dentro do raio, 22% fora (p_off_geofence)
        if random.random() < p_off_geofence:
            lat, lon = gen_point_near(GEOFENCE_CENTER, max_dist_m=GEOFENCE_RADIUS_M * 2.0)
        else:
            lat, lon = gen_point_near(GEOFENCE_CENTER, max_dist_m=GEOFENCE_RADIUS_M * 0.7)

        vel = max(0, round(random.gauss(15, 12), 1))
        bateria = max(0, min(100, int(random.gauss(70, 20))))
        estado = "EM_USO" if vel > 1 else random.choice(["PARADA", "MANUTENCAO"])

        ts = (now + dt.timedelta(seconds=i*5)).strftime("%Y-%m-%dT%H:%M:%SZ")
        item = {
            "placa": placa,
            "ts": ts,
            "lat": round(lat, 6),
            "lon": round(lon, 6),
            "vel": vel,
            "bateria": bateria,
            "status": estado,
            "sinais": {"ignicao": vel > 0, "queda": False}
        }
        out.append(item)
    return out

dataset = gen_telemetria(N_REGISTROS, P_OFF_GEOFENCE)

# Salva NDJSON
with PATH_NDJSON.open("w", encoding="utf-8") as f:
    for row in dataset:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

df = pd.DataFrame(dataset)
display(df.head(8))
print(f"NDJSON salvo em: {PATH_NDJSON}  | Registros:", len(df))



## 4) Engine de Regras (Geofence, Bateria Baixa, Inatividade)
Gera **alertas** com base na telemetria. Exporta **NDJSON** e exibe amostras.


In [ ]:

# @title Avaliar regras e exportar alertas
import pandas as pd, numpy as np

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))

def avaliar_regras(df):
    alerts = []
    df_sorted = df.sort_values("ts")
    # Inatividade: considera janela de 8 minutos (para demo)
    INATIVIDADE_MIN = 8

    last_movement = {}

    for row in df_sorted.itertuples():
        dist = haversine_m(row.lat, row.lon, GEOFENCE_CENTER[0], GEOFENCE_CENTER[1])
        # Fora de geofence
        if dist > GEOFENCE_RADIUS_M:
            alerts.append({
                "placa": row.placa, "tipo": "FORA_GEOFENCE",
                "ts": row.ts, "dist_m": round(dist, 1)
            })
        # Bateria baixa
        if row.bateria < 20:
            alerts.append({
                "placa": row.placa, "tipo": "BATERIA_BAIXA",
                "ts": row.ts, "bateria": int(row.bateria)
            })
        # Inatividade prolongada
        ts_dt = dt.datetime.strptime(row.ts, "%Y-%m-%dT%H:%M:%SZ")
        if row.vel > 0:
            last_movement[row.placa] = ts_dt
        else:
            last = last_movement.get(row.placa, None)
            if last is not None:
                delta_min = (ts_dt - last).total_seconds()/60
                if delta_min >= INATIVIDADE_MIN:
                    alerts.append({
                        "placa": row.placa, "tipo": "INATIVIDADE_PROLONGADA",
                        "ts": row.ts, "min_sem_mov": round(delta_min, 1)
                    })
                    # Evita spam
                    last_movement[row.placa] = ts_dt

    return pd.DataFrame(alerts)

df_alertas = avaliar_regras(df)
display(df_alertas.head(10))
print("Total de alertas:", len(df_alertas))

# Exporta NDJSON
with PATH_ALERTAS.open("w", encoding="utf-8") as f:
    for rec in df_alertas.to_dict(orient="records"):
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"Alertas NDJSON salvo em: {PATH_ALERTAS}")



## 5) Visualização – Mapa (Folium) + Dispersão (Matplotlib)
- Gera um **mapa interativo** com marcadores por **status** (cores).
- Salva HTML do mapa para compartilhar como evidência.


In [ ]:

# @title Criar mapa Folium e gráfico de dispersão
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt

# Cores por status (legenda)
COLOR_BY_STATUS = {
    "EM_USO": "blue",
    "PARADA": "gray",
    "MANUTENCAO": "orange",
    "ALERTA": "red",
    "OFFLINE": "purple"
}

# Seleciona último ponto por placa (estado "atual")
df_last = df.sort_values("ts").groupby("placa").tail(1).reset_index(drop=True)

# Se tiver alerta para a placa no último ts, pintar como ALERTA
# (regra simples para evidência)
alert_placas = set(df_alertas["placa"].unique().tolist()) if len(df_alertas) else set()
df_last["status_plot"] = df_last.apply(
    lambda r: "ALERTA" if r["placa"] in alert_placas else r["status"],
    axis=1
)

m = folium.Map(location=[GEOFENCE_CENTER[0], GEOFENCE_CENTER[1]], zoom_start=16)
folium.Circle(
    radius=GEOFENCE_RADIUS_M, location=GEOFENCE_CENTER,
    color="green", fill=False, weight=2, tooltip="Geofence (raio)"
).add_to(m)

cluster = MarkerCluster().add_to(m)

for row in df_last.itertuples():
    color = COLOR_BY_STATUS.get(row.status_plot, "blue")
    folium.Marker(
        location=[row.lat, row.lon],
        tooltip=f"{row.placa} • {row.status_plot} • bateria {row.bateria}%",
        icon=folium.Icon(color=color, icon="motorcycle", prefix="fa")
    ).add_to(cluster)

m.save(str(PATH_MAP_HTML))
print("Mapa salvo em:", PATH_MAP_HTML)

# Dispersão (proxy de mapa)
plt.figure(figsize=(5,5))
plt.scatter(df["lon"], df["lat"], alpha=0.4)
plt.title("Dispersão das posições (proxy de mapa)")
plt.xlabel("Longitude"); plt.ylabel("Latitude")
plt.show()



## 6) Mock de SSE (opcional – demonstração de tempo real)
Cria um **servidor local** de SSE em background e envia eventos com base nos alertas gerados.  
Você pode **consumir via cURL** ou com um **EventSource** no front.

> **Nota**: Em alguns ambientes Colab, conexões HTTP locais podem não funcionar. Use apenas como demo.


In [ ]:

# @title Iniciar Mock SSE (Flask) em background
import threading, time
from flask import Flask, Response

app = Flask(__name__)

def sse_stream():
    # Emite um evento por segundo com base no df_alertas
    for rec in df_alertas.to_dict(orient="records"):
        payload = json.dumps(rec, ensure_ascii=False)
        yield f"event: alerta\n"
        yield f"data: {payload}\n\n"
        time.sleep(1)
    # Mantém conexão viva mais um pouco
    for _ in range(3):
        yield "event: keepalive\n"
        yield "data: ok\n\n"
        time.sleep(1)

@app.route('/realtime/stream')
def stream():
    return Response(sse_stream(), mimetype='text/event-stream')

def run_app():
    app.run(host="0.0.0.0", port=8008, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()
print("Servidor SSE em http://127.0.0.1:8008/realtime/stream (pode não funcionar no Colab).")


In [ ]:

# @title Testar consumo via cURL (pode não funcionar no Colab)
import os, sys, subprocess, textwrap, shutil, time
cmd = ["bash", "-lc", "curl -N http://127.0.0.1:8008/realtime/stream --max-time 5 || true"]
print("Executando cURL por 5s...")
subprocess.run(cmd)
print("Teste finalizado.")



## 7) Exportar Evidências (ZIP)
Gera um ZIP com NDJSON, mapa HTML e um README curto para anexar na entrega.


In [ ]:

# @title Gerar ZIP de evidências
from zipfile import ZipFile, ZIP_DEFLATED

zip_path = OUT_DIR / "evidencias_mototrack.zip"
with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
    for p in [PATH_NDJSON, PATH_ALERTAS, PATH_MAP_HTML]:
        if p.exists():
            z.write(p, arcname=p.name)
    # README minimal
    readme_text = f"""
MotoTrack – Evidências (geradas pelo Colab)
Itens:
- telemetria.ndjson
- alertas.ndjson
- mapa_mototrack.html

Como demonstrar no vídeo:
1) Mostrar DataFrame inicial (telemetria) e o NDJSON.
2) Mostrar DataFrame de alertas e o NDJSON.
3) Abrir o HTML do mapa (download) e destacar pins/cores.
4) (Opcional) Rodar Mock SSE e fazer cURL.
"""
    z.writestr("README.txt", readme_text.strip())

print("ZIP gerado em:", zip_path)



## 8) Checklist de Avaliação
- ✅ **Fluxo completo** (captura simulada → regras → visualização → arquivos NDJSON/HTML).  
- ✅ **Dashboard/Interface**: mapa com **cores por estado** + **legenda** e dispersão.  
- ✅ **Alertas** gerados (FORA_GEOFENCE, BATERIA_BAIXA, INATIVIDADE_PROLONGADA).  
- 🟡 **Tempo real (SSE)**: mock local (pode falhar no Colab; use como evidência didática).  
- 🟡 **Integração completa**: se precisar, complemente com Swagger da API .NET e compose.

> Dúvidas? Rode tudo em ordem e gere o ZIP de evidências.
